### Set up

In [2]:
# !pip install wrds

import wrds

conn = wrds.Connection(wrds_username = 'xqzhang')
#conn.create_pgpass_file()

Loading library list...
Done


In [210]:
import numpy as np
import pandas as pd

### View column names

In [3]:
# all columns in a specific table, which include all stocks' data in one day

# conn.describe_table(library = 'taqmsec', table = 'cqm_20180608')

### Get data

In [41]:
# WRDS data is stored on Postgres.
# SELECT => pick variable(s), * for all
# FROM => dataset name
# other SQL sytex including, WHERE, ORDER BY, GROUP BY, amd so on

# stmt: SELCT column1, column2, ...
#       From table_name
#       WHERE symbol and time range

stmt = "SELECT date, time_m, ex, sym_root, bid, ask, bidsiz, asksiz \
        FROM taqmsec.cqm_20180608 \
        WHERE sym_root = 'AAPL' AND (time_m BETWEEN '04:00:00' AND '04:05:00') "
data =  conn.raw_sql(stmt)

,date,time_m,ex,sym_root,bid,ask,bidsiz,asksiz
0,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,1.0,0.0
1,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,2.0,0.0
2,2018-06-08,04:00:00.072000,P,AAPL,15.00,0.0,1.0,0.0
3,2018-06-08,04:00:00.072000,P,AAPL,190.00,0.0,2.0,0.0
4,2018-06-08,04:00:00.072000,P,AAPL,190.31,0.0,8.0,0.0


### Calculate second volume and average second price

In [361]:
data.columns = ['date', 'time','ex','ticker','bid','ask','bidsiz','asksiz']
data.head()

,date,time,ex,ticker,bid,ask,bidsiz,asksiz
0,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,1.0,0.0
1,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,2.0,0.0
2,2018-06-08,04:00:00.072000,P,AAPL,15.00,0.0,1.0,0.0
3,2018-06-08,04:00:00.072000,P,AAPL,190.00,0.0,2.0,0.0
4,2018-06-08,04:00:00.072000,P,AAPL,190.31,0.0,8.0,0.0


In [362]:
a = data.copy()

In [363]:
a.head()

,date,time,ex,ticker,bid,ask,bidsiz,asksiz
0,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,1.0,0.0
1,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,2.0,0.0
2,2018-06-08,04:00:00.072000,P,AAPL,15.00,0.0,1.0,0.0
3,2018-06-08,04:00:00.072000,P,AAPL,190.00,0.0,2.0,0.0
4,2018-06-08,04:00:00.072000,P,AAPL,190.31,0.0,8.0,0.0


In [364]:
a['hour'] = 0
a['minute'] = 0
a['second'] = 0

In [365]:
for i in range(len(a)):
    a.loc[i, 'hour'] = a.loc[i].time.hour
    a.loc[i, 'minute'] = a.loc[i].time.minute
    a.loc[i, 'second'] = a.loc[i].time.second

In [348]:
a.head()

,date,time,ex,ticker,bid,ask,bidsiz,asksiz,hour,minute,second
0,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,1.0,0.0,4,0,0
1,2018-06-08,04:00:00.072000,P,AAPL,1.00,0.0,2.0,0.0,4,0,0
2,2018-06-08,04:00:00.072000,P,AAPL,15.00,0.0,1.0,0.0,4,0,0
3,2018-06-08,04:00:00.072000,P,AAPL,190.00,0.0,2.0,0.0,4,0,0
4,2018-06-08,04:00:00.072000,P,AAPL,190.31,0.0,8.0,0.0,4,0,0


In [349]:
size = a.groupby([a.date, a.hour, a.minute, a.second]).sum().drop(['bid', 'ask'], axis=1)

In [358]:
price = a.groupby([a.date, a.hour, a.minute, a.second]).mean().drop(['bid', 'ask'], axis=1)

In [359]:
size.head()

bid      ask  bidsiz  asksiz
date       hour minute second                                  
2018-06-08 4    0      0       4215.84  3856.84    57.0    94.0
                       5        571.86   574.04     3.0     5.0
                       7        381.24   382.79     2.0     2.0
                       8        190.62   191.35     1.0     2.0
                       10       380.40   191.35     4.0     2.0

In [360]:
price.head()

bidsiz    asksiz
date       hour minute second                  
2018-06-08 4    0      0         2.28  3.760000
                       5         1.00  1.666667
                       7         1.00  1.000000
                       8         1.00  2.000000
                       10        2.00  1.000000